In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/23 16:30:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/23 16:30:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
! wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet


--2024-02-23 16:33:07--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.155.128.46, 18.155.128.187, 18.155.128.222, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.155.128.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 308924937 (295M) [application/x-www-form-urlencoded]
Saving to: ‘fhvhv_tripdata_2021-01.parquet’

fhvhv_tripdata_2021 100%[===================>] 294.61M   186MB/s    in 1.6s    

2024-02-23 16:33:08 (186 MB/s) - ‘fhvhv_tripdata_2021-01.parquet’ saved [308924937/308924937]



In [10]:
df = spark.read.format("Parquet").load("fhvhv_tripdata_2021-01.parquet")

In [4]:
df.show()

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

In [5]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp_ntz (nullable = true)
 |-- on_scene_datetime: timestamp_ntz (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- dropoff_datetime: timestamp_ntz (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_ride_f

In [12]:
df = df.repartition(8)

In [13]:
df.rdd.getNumPartitions()

8

In [18]:
df.write.format("PARQUET").mode("overwrite").option('path','fhvhv/2021/01').save()

In [16]:
! ls -lh fhvhv/2021/01

total 479M
-rw-r--r-- 1 pedroh pedroh   0 Feb 23 17:36 _SUCCESS
-rw-r--r-- 1 pedroh pedroh 60M Feb 23 17:36 part-00000-bcb495f9-a5dd-4e93-b1a2-34bc8c8cd025-c000.snappy.parquet
-rw-r--r-- 1 pedroh pedroh 60M Feb 23 17:36 part-00001-bcb495f9-a5dd-4e93-b1a2-34bc8c8cd025-c000.snappy.parquet
-rw-r--r-- 1 pedroh pedroh 60M Feb 23 17:36 part-00002-bcb495f9-a5dd-4e93-b1a2-34bc8c8cd025-c000.snappy.parquet
-rw-r--r-- 1 pedroh pedroh 60M Feb 23 17:36 part-00003-bcb495f9-a5dd-4e93-b1a2-34bc8c8cd025-c000.snappy.parquet
-rw-r--r-- 1 pedroh pedroh 60M Feb 23 17:36 part-00004-bcb495f9-a5dd-4e93-b1a2-34bc8c8cd025-c000.snappy.parquet
-rw-r--r-- 1 pedroh pedroh 60M Feb 23 17:36 part-00005-bcb495f9-a5dd-4e93-b1a2-34bc8c8cd025-c000.snappy.parquet
-rw-r--r-- 1 pedroh pedroh 60M Feb 23 17:36 part-00006-bcb495f9-a5dd-4e93-b1a2-34bc8c8cd025-c000.snappy.parquet
-rw-r--r-- 1 pedroh pedroh 60M Feb 23 17:36 part-00007-bcb495f9-a5dd-4e93-b1a2-34bc8c8cd025-c000.snappy.parquet


In [19]:
df = spark.read.parquet('fhvhv/2021/01')

In [20]:
df

DataFrame[hvfhs_license_num: string, dispatching_base_num: string, originating_base_num: string, request_datetime: timestamp_ntz, on_scene_datetime: timestamp_ntz, pickup_datetime: timestamp_ntz, dropoff_datetime: timestamp_ntz, PULocationID: bigint, DOLocationID: bigint, trip_miles: double, trip_time: bigint, base_passenger_fare: double, tolls: double, bcf: double, sales_tax: double, congestion_surcharge: double, airport_fee: double, tips: double, driver_pay: double, shared_request_flag: string, shared_match_flag: string, access_a_ride_flag: string, wav_request_flag: string, wav_match_flag: string]

In [22]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID').where("hvfhs_license_num = 'HV0003'").show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-05 12:42:09|2021-01-05 13:04:44|          22|         261|
|2021-01-01 10:41:59|2021-01-01 11:02:42|         122|         196|
|2021-01-02 08:12:07|2021-01-02 08:42:41|         162|           1|
|2021-01-02 14:16:06|2021-01-02 14:22:35|         189|         181|
|2021-01-26 19:34:12|2021-01-26 19:47:11|         126|          69|
|2021-01-06 08:26:45|2021-01-06 08:53:00|          56|          75|
|2021-01-07 23:17:27|2021-01-07 23:30:24|         152|         159|
|2021-01-08 21:46:23|2021-01-08 21:55:12|         136|         174|
|2021-01-28 01:07:33|2021-01-28 01:17:39|         169|          69|
|2021-01-08 13:56:20|2021-01-08 14:00:04|          89|          89|
|2021-01-17 09:00:11|2021-01-17 09:13:05|         169|          69|
|2021-01-30 16:27:04|2021-01-30 16:42:19|       

In [27]:
from pyspark.sql import functions as F
from pyspark.sql import types

In [24]:
df = df.withColumn('pickup_date', F.to_date('pickup_datetime')) \
    .withColumn('dropoff_date', F.to_date('dropoff_datetime'))

In [26]:
df.select('pickup_date', 'dropoff_date').show()

+-----------+------------+
|pickup_date|dropoff_date|
+-----------+------------+
| 2021-01-05|  2021-01-05|
| 2021-01-01|  2021-01-01|
| 2021-01-02|  2021-01-02|
| 2021-01-02|  2021-01-02|
| 2021-01-24|  2021-01-24|
| 2021-01-26|  2021-01-26|
| 2021-01-06|  2021-01-06|
| 2021-01-07|  2021-01-07|
| 2021-01-08|  2021-01-08|
| 2021-01-28|  2021-01-28|
| 2021-01-31|  2021-01-31|
| 2021-01-08|  2021-01-08|
| 2021-01-17|  2021-01-17|
| 2021-01-30|  2021-01-30|
| 2021-01-04|  2021-01-04|
| 2021-01-29|  2021-01-29|
| 2021-01-21|  2021-01-21|
| 2021-01-01|  2021-01-01|
| 2021-01-14|  2021-01-14|
| 2021-01-28|  2021-01-28|
+-----------+------------+
only showing top 20 rows



In [29]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    else:
        return f'e/{num:03x}'

In [30]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [31]:
df.withColumn('pickup_date', F.to_date('pickup_datetime')) \
    .withColumn('dropoff_date', F.to_date('dropoff_datetime')) \
    .withColumn('base_id', crazy_stuff_udf('dispatching_base_num')) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID').show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/acc| 2021-01-05|  2021-01-05|          22|         261|
|  e/b37| 2021-01-01|  2021-01-01|         122|         196|
|  s/acd| 2021-01-02|  2021-01-02|         162|           1|
|  e/b33| 2021-01-02|  2021-01-02|         189|         181|
|  e/9ce| 2021-01-24|  2021-01-24|          25|          36|
|  s/b44| 2021-01-26|  2021-01-26|         126|          69|
|  s/acd| 2021-01-06|  2021-01-06|          56|          75|
|  e/b3c| 2021-01-07|  2021-01-07|         152|         159|
|  e/b3f| 2021-01-08|  2021-01-08|         136|         174|
|  s/acd| 2021-01-28|  2021-01-28|         169|          69|
|  e/9ce| 2021-01-31|  2021-01-31|          41|          80|
|  e/b32| 2021-01-08|  2021-01-08|          89|          89|
|  e/b3b| 2021-01-17|  2021-01-17|         169|          69|
|  e/b3f| 2021-01-30|  2